####Copyright https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans

In [ ]:
import torch
import numpy as np

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')



In [ ]:
y = torch.from_numpy(targets)
x = torch.from_numpy(inputs)
print("inputs:", inputs)
print("targets:", targets)

inputs: [[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]
targets: [[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]


In [ ]:
#Initilizing the weights
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print("weights w: ", w)
print("bias b: ",b)

weights w:  tensor([[ 1.8842,  0.6396,  0.1245],
        [ 0.4543, -0.1152, -1.5338]], requires_grad=True)
bias b:  tensor([-0.4687,  0.1853], requires_grad=True)


In [ ]:
#Defining the model
def model(inputs):
  model = inputs @ w.t() + b
  return model

In [ ]:

pred = model(x)

In [ ]:

#Definging the mean square error
def mse(t1,t2):
  diff = t1 - t2 
  return torch.sum(diff * diff) / diff.numel()

In [ ]:
#Computing the loss 
loss = mse(pred, y)
print("loss: ", loss)

loss:  tensor(12688.3506, grad_fn=<DivBackward0>)


In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [ ]:
#Compute the gradients
loss.backward()
w.grad
print(w.grad)

In [ ]:
#First step is to predict the output
pred = model(x)
print(pred)

tensor([[185.2822, -40.3220],
        [235.2436, -66.7736],
        [256.3803, -64.6897],
        [223.8258, -15.1773],
        [199.6559, -86.8939]], grad_fn=<AddBackward0>)


In [ ]:
#Second step is to compute the loss
loss = mse(pred, y)
print(loss)

tensor(23396.9199, grad_fn=<DivBackward0>)


In [ ]:
#Third step is to compute the gradient
loss.backward()


In [ ]:
#Forth step is to adjust the weights to minimize the loss
with torch.no_grad():
  w -= w.grad * 1e-6
  b -= b.grad * 1e-6
  w.grad.zero_()
  b.grad.zero_()
  print("w:", w)
  print("b:", b) 

w: tensor([[ 1.8716,  0.6279,  0.1170],
        [ 0.4664, -0.1011, -1.5251]], requires_grad=True)
b: tensor([-0.4688,  0.1854], requires_grad=True)


In [ ]:
#Create the loss
preds =  model(x)
loss = mse(preds, y)
print(loss)

tensor(22633.7012, grad_fn=<DivBackward0>)


In [ ]:
#Train the model for 100 epochs
for i in range(3000):
  preds = model(x)
  loss = mse(preds, y)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
preds = model(x)
loss = mse(preds, y)

In [ ]:
loss

tensor(0.5143, grad_fn=<DivBackward0>)

In [ ]:
preds

tensor([[ 57.0862,  70.3461],
        [ 82.2401, 100.6110],
        [118.7428, 133.0342],
        [ 21.1020,  37.0339],
        [101.8709, 119.0622]], grad_fn=<AddBackward0>)

In [ ]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

#### Linear Regression using PyTorch

In [ ]:
import torch.nn as nn

In [ ]:
#Input (temp, rainfall, humidity)
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:

train_ds = TensorDataset(inputs, targets)
train_ds

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 68.,  96.,  71.],
        [ 87., 134.,  58.],
        [ 68.,  97.,  70.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [104., 118.],
        [119., 133.],
        [102., 120.]])


In [ ]:
print(xb)
print(yb)

tensor([[ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 68.,  96.,  71.],
        [ 87., 134.,  58.],
        [ 68.,  97.,  70.]])
tensor([[ 56.,  70.],
        [ 22.,  37.],
        [104., 118.],
        [119., 133.],
        [102., 120.]])


In [ ]:
 #Define a model
 model = nn.Linear(3,2)
 print(model.weight)
 print(model.bias)

Parameter containing:
tensor([[ 0.2163, -0.1726, -0.5025],
        [ 0.2681,  0.2311, -0.1461]], requires_grad=True)
Parameter containing:
tensor([-0.4835, -0.0126], requires_grad=True)


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2163, -0.1726, -0.5025],
         [ 0.2681,  0.2311, -0.1461]], requires_grad=True),
 Parameter containing:
 tensor([-0.4835, -0.0126], requires_grad=True)]

In [ ]:
#Generate prediction
preds = model(inputs)

In [ ]:
preds

tensor([[-17.8637,  28.7593],
        [-28.1467,  35.3702],
        [-33.9378,  45.8035],
        [ -4.4329,  31.8646],
        [-37.3014,  30.4443],
        [-17.4748,  28.7963],
        [-28.4765,  34.9930],
        [-34.2240,  45.9255],
        [ -4.8219,  31.8276],
        [-38.0201,  30.0302],
        [-18.1936,  28.3822],
        [-27.7578,  35.4072],
        [-33.6080,  46.1806],
        [ -3.7142,  32.2788],
        [-37.6903,  30.4073]], grad_fn=<AddmmBackward0>)

In [ ]:
#Loss Funtioin

In [ ]:
#Import nn.Funtional
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)

In [ ]:
#Define the optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-6)

#Train the model

In [ ]:
#Utility funtion to traint the model
def fit(num_epoch, model, loss_fn, opt, train_dl):
  for epoch in range(num_epoch):
    for xb, yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred, yb)
      loss.backward()
      opt.step()
      opt.zero_grad()

    if (epoch + 1) % 10 == 0:
      print("Epoch [{}/{}], Loss {:.4f}".format(epoch+1, num_epoch, loss.item()))


In [ ]:
fit(500, model, loss_fn, opt, train_dl)

Epoch [10/500], Loss 2.7988
Epoch [20/500], Loss 3.9523
Epoch [30/500], Loss 1.4115
Epoch [40/500], Loss 2.0230
Epoch [50/500], Loss 1.9581
Epoch [60/500], Loss 2.1124
Epoch [70/500], Loss 1.3041
Epoch [80/500], Loss 1.7977
Epoch [90/500], Loss 3.7035
Epoch [100/500], Loss 1.8377
Epoch [110/500], Loss 2.6088
Epoch [120/500], Loss 4.8728
Epoch [130/500], Loss 4.2444
Epoch [140/500], Loss 2.1049
Epoch [150/500], Loss 2.1857
Epoch [160/500], Loss 2.3819
Epoch [170/500], Loss 0.9827
Epoch [180/500], Loss 4.9523
Epoch [190/500], Loss 2.4862
Epoch [200/500], Loss 3.4070
Epoch [210/500], Loss 3.9900
Epoch [220/500], Loss 5.5963
Epoch [230/500], Loss 3.9120
Epoch [240/500], Loss 1.3716
Epoch [250/500], Loss 1.0176
Epoch [260/500], Loss 1.6248
Epoch [270/500], Loss 3.8638
Epoch [280/500], Loss 1.3518
Epoch [290/500], Loss 1.4967
Epoch [300/500], Loss 1.9119
Epoch [310/500], Loss 4.0930
Epoch [320/500], Loss 2.3948
Epoch [330/500], Loss 2.7619
Epoch [340/500], Loss 4.4391
Epoch [350/500], Loss 1